<a href="https://colab.research.google.com/github/SharafatAhmed/AI-Agents/blob/main/A_Basic_FAQ_AI_Agent_using_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [2]:
!pip install ollama

In [3]:
!pip install ollama langchain langchain-community rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


rapidfuzz is used for fast fuzzy matching of user query to answer FAQ questions.

langchain-community provides the ChatOllama wrapper used below

In [4]:
import json

faq = [
    {
        "question": "What is EWUDA?",
        "answer": "EWUDA (East West University Department Assistant) is a virtual assistant designed to answer student FAQs 24/7."
    },
    {
        "question": "What does EWUDA do?",
        "answer": "EWUDA helps students by answering common department questions such as course advising, scholarships, office information, and reminders."
    },
    {
        "question": "When is the department officer available?",
        "answer": "The department officer is available from 9:00 AM to 6:00 PM during office hours. Outside this time, EWUDA can help with FAQs."
    },
    {
        "question": "I am a new student. Where can I check my advisor name?",
        "answer": "You can check your advisor’s name in your student portal under the profile section."
    },
    {
        "question": "What should I do if I didn’t get a course?",
        "answer": "You should contact your advisor by email or visit during their office hours to resolve the issue."
    },
    {
        "question": "How do I contact my advisor?",
        "answer": "You can find your advisor’s email and office room number in the student portal under advisor information."
    },
    {
        "question": "Where is the department office located?",
        "answer": "The department office is located on the 5th floor, Room 507, of the main academic building."
    },
    {
        "question": "How can I apply for a scholarship?",
        "answer": "You need to fill out the scholarship application form available on the university website and submit it to the financial aid office before the deadline."
    },
    {
        "question": "What documents are required for scholarship application?",
        "answer": "You usually need your transcript, financial documents, and a completed scholarship form. Check the latest notice for specific requirements."
    },
    {
        "question": "How does EWUDA help with emails?",
        "answer": "EWUDA can generate template replies for common student queries and assist in sending bulk updates to students."
    },
    {
        "question": "Can EWUDA send reminders for meetings?",
        "answer": "Yes, EWUDA can schedule and send reminders to faculty and students about meetings or important deadlines."
    },
    {
        "question": "How does EWUDA prioritize emails?",
        "answer": "EWUDA uses keywords like 'deadline' or 'urgent' along with sender type to sort emails into urgent and normal categories."
    },
    # Non-university / irrelevant questions: EWUDA will refuse these
    {
        "question": "What's the weather today?",
        "answer": "Sorry — EWUDA only answers university or department-related questions. For weather information, please use a weather app or website."
    },
    {
        "question": "Who won the cricket match yesterday?",
        "answer": "EWUDA does not provide sports or general news updates. Please check a sports news website or app for match results."
    },
    {
        "question": "Can you give me medical advice for a headache?",
        "answer": "EWUDA cannot provide medical advice. For health concerns, contact a qualified healthcare professional or campus health services."
    },
    {
        "question": "Recommend a movie to watch tonight",
        "answer": "EWUDA only helps with department and university-related matters. For entertainment recommendations, please try a movie/streaming service or entertainment forum."
    }
]

with open("faq.json", "w", encoding="utf-8") as f:
    json.dump(faq, f, indent=2, ensure_ascii=False)

print("faq.json written with", len(faq), "entries for EWUDA assistant.")


faq.json written with 16 entries for EWUDA assistant.


In [5]:
import json
from rapidfuzz import process, fuzz

# Load FAQ from JSON file created in Cell 3
with open("faq.json", "r", encoding="utf-8") as f:
    faq = json.load(f)

questions = [q["question"] for q in faq]

# Retrieval function: top-k fuzzy matches and scores
def get_top_matches(user_query, k=3, scorer=fuzz.QRatio):
    # returns list of tuples (matched_question, score, index)
    results = process.extract(user_query, questions, scorer=scorer, limit=k)
    return results

# Build an in-context prompt from top matched examples
def build_icl_prompt(
    examples,
    user_question,
    instruction=(
        "You are EWUDA (East West University Department Assistant), a helpful assistant "
        "that answers student FAQs 24/7. Always stay polite and concise.\n"
        "Answer the user question based on the examples. "
        "If none are relevant, reply exactly: "
        "'I’m EWUDA. I don’t know the answer to that. Please refine your question or "
        "contact the department officer during 9:00 AM – 6:00 PM office hours.'"
    )
):
    prompt = instruction.strip() + "\n\n"
    prompt += "Here are example FAQs:\n\n"
    for ex in examples:
        prompt += f"Q: {ex['question']}\nA: {ex['answer']}\n\n"
    prompt += f"User question: {user_question}\nA:"
    return prompt


In [6]:
import re

# List of irrelevant topic keywords (expand as needed)
irrelevant_keywords = [
    "weather", "food", "movie", "sports", "music", "football", "cricket",
    "politics", "game", "recipe", "song", "travel", "news", "shopping"
]

def is_irrelevant(query: str) -> bool:
    q = query.lower()
    return any(word in q for word in irrelevant_keywords)

def answer_question(user_query, threshold=60):
    # 1. Check if irrelevant first
    if is_irrelevant(user_query):
        return ("I’m EWUDA. I only answer questions related to East West University "
                "department matters (courses, advisors, scholarships, office info, etc.). "
                "For other topics, please use a general search engine.")

    # 2. Get matches from FAQ
    matches = get_top_matches(user_query, k=3)
    top_q, top_score, top_index = matches[0]

    if top_score < threshold:
        return ("I’m EWUDA. I don’t know the answer to that. "
                "Please refine your question or contact the department officer "
                "during 9:00 AM – 6:00 PM office hours.")

    # 3. Build prompt using top examples
    top_examples = [faq[m[2]] for m in matches]
    prompt = build_icl_prompt(top_examples, user_query)

    # Here we just simulate model output by selecting the closest FAQ answer
    return faq[top_index]["answer"]


In [7]:
# Test with relevant queries
print("Q: I am a new student, where do I check my advisor name?")
print("A:", answer_question("I am a new student, where do I check my advisor name?"))
print()

print("Q: What should I do if I didn’t get a course?")
print("A:", answer_question("What should I do if I didn’t get a course?"))
print()

# Test with irrelevant query
print("Q: What’s the weather today?")
print("A:", answer_question("What’s the weather today?"))
print()

# Test with weakly related / unknown query
print("Q: How can I apply for library membership?")
print("A:", answer_question("How can I apply for library membership?"))


Q: I am a new student, where do I check my advisor name?
A: You can check your advisor’s name in your student portal under the profile section.

Q: What should I do if I didn’t get a course?
A: You should contact your advisor by email or visit during their office hours to resolve the issue.

Q: What’s the weather today?
A: I’m EWUDA. I only answer questions related to East West University department matters (courses, advisors, scholarships, office info, etc.). For other topics, please use a general search engine.

Q: How can I apply for library membership?
A: You need to fill out the scholarship application form available on the university website and submit it to the financial aid office before the deadline.


In [10]:
print("Welcome to EWUDA (East West University Department Assistant)")
print("Type 'exit' to quit.\n")

while True:
    user_q = input("You: ")
    if user_q.lower().strip() in ["exit", "quit"]:
        print("EWUDA: Goodbye! Have a nice day.")
        break
    response = answer_question(user_q)
    print("EWUDA:", response)


Welcome to EWUDA (East West University Department Assistant)
Type 'exit' to quit.

You: whats the weather today?
EWUDA: I’m EWUDA. I only answer questions related to East West University department matters (courses, advisors, scholarships, office info, etc.). For other topics, please use a general search engine.
You: exit
EWUDA: Goodbye! Have a nice day.
